In [1]:
# Import statements
# import yfinance as yf
import pandas as pd
import os
import numpy as np
import datetime as dt
from datetime import *
from pathlib import Path
import pandas_ta as ta
from price_data_pull_yfinance import gethistoricalOHLC, saveHistStockData, loadHistDataFromDisk
from datetime import date
today = date.today()

from global_vars import dataPathToOHLC, pathToMasterDF, float16Cols, float32Cols

In [2]:
import yfinance as yf

In [3]:
yf.__version__

'0.1.67'

# Testing it for Bitcoin

In [4]:
ticker = yf.Ticker("BTC-USD")

# get stock info
print(ticker.info)

# start = dt.datetime(2018, 1, 1)
# end = today
# # get historical market data
# df = ticker.history(start=start,  end="2021-11-09", interval = "1m")


{'companyOfficers': [], 'name': 'Bitcoin', 'startDate': 1367107200, 'description': 'Bitcoin (BTC) is a cryptocurrency . Users are able to generate BTC through the process of mining. Bitcoin has a current supply of 18,882,875. The last known price of Bitcoin is 58,647.26190846 USD and is up 3.81 over the last 24 hours. It is currently trading on 8347 active market(s) with $34,798,046,427.24 traded over the last 24 hours. More information can be found at https://bitcoin.org/.', 'maxAge': 1, 'exchange': 'CCC', 'shortName': 'Bitcoin USD', 'exchangeTimezoneName': 'UTC', 'exchangeTimezoneShortName': 'UTC', 'isEsgPopulated': False, 'gmtOffSetMilliseconds': '0', 'quoteType': 'CRYPTOCURRENCY', 'symbol': 'BTC-USD', 'messageBoardId': 'finmb_BTC_CCC', 'market': 'ccc_market', 'previousClose': 53741.33, 'regularMarketOpen': 53741.33, 'twoHundredDayAverage': 45991.64, 'trailingAnnualDividendYield': None, 'payoutRatio': None, 'volume24Hr': 39052963840, 'regularMarketDayHigh': 55237.258, 'navPrice': No

In [ ]:
def pullMinuteData(ticker, timeframe = 4):
    '''function to pull a certain time frame of mintue date from yahoo finance as they only allow 1 week per request on that level of granularity.
    Inputs: 
        - ticker: a yfinance ticker object
        - timeframe: (int) in weeks
    Output:
        - df: a pandas dataframe with the requested data'''

    # generate a list of datetime tuples that have the requested intervals
    #df = ticker.history(start=start,  end="2021-11-09", interval = "1m")

    
    today = datetime.today()
    weeks = 3
    days = timedelta(7*weeks)
    startDate = today - days
    startDate = startDate.date()
    dfs = []
    currDate = today.date()
    while currDate > startDate:
        endIntervall = currDate
        startIntervall = (currDate - timedelta(6))
        df = ticker.history(start=startIntervall,  end=endIntervall)
        dfs.append(df)

In [8]:
today = datetime.today()

In [ ]:
spy = yf.Ticker("^GSPC")

# get stock info
spy.info

start = dt.datetime(2012, 1, 1)
end = today
# get historical market data
df = spy.history(start="2010-01-01",  end="2021-11-09")


ConnectionError: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/BTC-USD?period1=1637449200&period2=1637967600&interval=1d&includePrePost=False&events=div%2Csplits (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000012968F3E790>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
df['log_price'] = np.log(df.Close)
df['log_return'] = df['log_price'].diff()
print("Log Return", df.log_return.sum())
totalLogReturn = df.log_return.sum()
df = df.iloc[1: , :]

#### Sanity check if the returns make sense

In [ ]:
minDate = df.Date.min()
maxDate = df.Date.max()
V0 = df.loc[df.Date == minDate].Close.iloc[0]
Vt = df.loc[df.Date == maxDate].Close.iloc[0]
print(V0, Vt)

In [ ]:
Vt_calc = V0 * np.exp(totalLogReturn)
Vt_calc

In [ ]:
df

### Lets take a look how much money you would have made if you would have predicted perfectly on a day to day basis

Trading idea with ideal knowledge of the future one day returns, lets see how much one could make

In [ ]:
df['signal'] = np.where(df['log_return']>=0, 1, -1)
df

In [ ]:
def rowWiseBacktest(df, startcap = 10000):
    '''Takes a dataframe with close prices and itereates over them. Calculates the gains realized or lost via the trade signal'''
    holding = 0 # amount of shares
    cap = startcap 
    returns, purchase_price, div = 0, 0, 0

    first, second = 0,0
    for idx, row in df.iterrows():
        # print(row)
        if (holding == 0) & (row.signal == 1) & (cap >0) : # we anticipate gains, we want to buy
            div = 0
            holding = cap/row.Close   # buy all shares with all the money
            cap = 0 
            returns += row.log_return
            purchase_price = row.Close
            div += row.Dividends
            first += 1
        elif ((holding > 0) & (row.signal == -1)): # ew anticipate losses, we want to sell OR we reached the end of the backtest. In this case we also want to sell 
            second += 1
            # print("Second Condidtion")
            # print('before exectuion:')
            # print(holding, cap, returns, purchase_price, div)
            cap = (purchase_price + ((purchase_price*returns)+div))*holding
            holding = 0

            # print(holding, cap, returns, purchase_price, div)
            # print("####################################################################")
        else:
            pass  
        if idx == df.shape[0]:
            print("ran until:", idx) 

    final_return = np.log(cap)- np.log(startcap)
    return cap, final_return, holding
        
    

In [ ]:
cap, final_return, holding = rowWiseBacktest(df)
print("Capital after the backtesting period", cap)
print("Final log-return after backtest period", final_return)
print("Final holding after backtest period", holding)

In [ ]:
totalLogReturn_right = df[df.log_return>=0]['log_return'].sum() 

print(totalLogReturn_right)
Vt_calc = V0 * np.exp(totalLogReturn_right)
Vt_calc

In [ ]:
gain = df.loc[df.Date == maxDate].Close.iloc[0] - df.loc[df.Date == minDate].Close.iloc[0] 
dividendes = df.Dividends.sum()
totalGain = gain + dividendes
holdingPeriod = ((maxDate-minDate).days/365)
totalReturn = totalGain / df.loc[df.Date == minDate].Close.iloc[0] *100
print("Total Return: ", totalReturn)
annualizedReturn = totalReturn/ ((maxDate-minDate).days/365)
print("annualizedReturn", annualizedReturn)

In [ ]:
minDate = df.Date.min()
maxDate = df.Date.max()

df_Return = 100*()/df.loc[df.Date == minDate].Close.iloc[0]
annualizedReturn = df_Return/ ((maxDate-minDate).days/365)
print("annualized Return:", annualizedReturn)

In [ ]:
# Calculating anualized Returns
annualizedReturns = df_Return/((maxDate-minDate).days/365)


In [ ]:
df.loc[df.date == minDate].close.iloc[0]


In [ ]:
minDate

In [ ]:
(dt.date(2021,11,15)-dt.date(2005, 1, 1)).days/365

In [ ]:
1.363 / 16.882191780821916